In [28]:
import re
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from PIL import Image

import os
from os import listdir
from os.path import isfile, join
import string

from load_data import load_images
from descriptors.hog import Hog_descriptor

In [9]:
train_images, train_labels, test_images, test_labels = load_images('ufi-cropped')

In [10]:
train_images.shape, test_images.shape

((4316, 128, 128), (605, 128, 128))

In [21]:
train_raw, test_raw = np.copy(train_images), np.copy(test_images)
train_raw.shape, test_raw.shape

((4316, 128, 128), (605, 128, 128))

In [20]:
train_hog, test_hog = np.copy(train_images), np.copy(test_images)
for i in range(train_images.shape[0]):
    hog = Hog_descriptor(train_raw[i], cell_size=8, bin_size=8)
    _ , train_hog[i] = hog.extract()
for i in range(test_images.shape[0]):
    hog = Hog_descriptor(test_raw[i], cell_size=8, bin_size=8)
    _ , test_hog[i] = hog.extract()
    
train_raw.shape, test_hog.shape

In [5]:
train_images = train_images.reshape(-1, 1, 128, 128)
test_images = test_images.reshape(-1, 1, 128, 128)

train_images.shape, test_images.shape

((4316, 1, 128, 128), (605, 1, 128, 128))

In [33]:
class UFIDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img = self.images[index]
        image = torch.from_numpy(img).float()
        label = self.labels[index]
        
        return image, label

In [34]:
ufi_train_data = UFIDataset(train_images, train_labels)

In [35]:
train_loader = torch.utils.data.DataLoader(dataset=ufi_train_data,
                                         batch_size=64,
                                         shuffle=True)

In [36]:
ufi_test_data = UFIDataset(test_images, test_labels)

In [37]:
test_loader = torch.utils.data.DataLoader(dataset=ufi_test_data,
                                         batch_size=64,
                                         shuffle=True)

In [38]:
#sample batch check
data_iter = iter(train_loader)
X, Y = data_iter.next()

X.size(), Y.size()

(torch.Size([64, 3, 128, 128]), torch.Size([64]))

In [39]:
## the CNN model

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
os.environ['TORCH_MODEL_ZOO'] = 'E:\\College\\Final Year Project\\ufi\\models'
model = torchvision.models.resnet50(pretrained=True)

In [42]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [43]:
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(8192, 2048)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(0.5)),
                          ('fc2', nn.Linear(2048, 1024)),
                          ('relu2', nn.ReLU()),
                          ('dropout2', nn.Dropout(0.5)),
                          ('output', nn.Linear(1024, 605))
                          ])) 
model.avgpool = nn.AvgPool2d(kernel_size=3, stride=1, padding=0)
model.fc = classifier

In [45]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.01)

In [47]:
epochs = 1000
steps = 0

train_losses, test_losses, accuracies = [], [], []
for e in range(epochs):
    for images, labels in train_loader:
        running_loss = 0
        images, labels = images.cuda(), labels.cuda()
        steps += 1
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    
        ## validation pass
        if steps % 10 == 0:
            test_loss = 0
            accuracy = 0
            
            model.eval()
            with torch.no_grad():
                for images, labels in test_loader:
                    images, labels = images.cuda(), labels.cuda()

                    log_ps = model(images)

                    test_loss += criterion(log_ps, labels)

                    ps = torch.exp(log_ps)
                    _, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.cuda.FloatTensor))
                    
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(test_loader))
            accuracies.append(accuracy/len(test_loader))

            print("Epoch [{}/{}] ".format(e+1, epochs),
                  "Training Loss:.. {:.3f}".format(train_losses[-1]),
                  "Test Loss:.. {:.3f}".format(test_losses[-1]),
                  "Test Accuracy:.. {:.3f}".format(accuracies[-1]))
        
            model.train()

Epoch [1/1000]  Training Loss:.. 0.096 Test Loss:.. 6.510 Test Accuracy:.. 0.003
Epoch [1/1000]  Training Loss:.. 0.098 Test Loss:.. 6.420 Test Accuracy:.. 0.000
Epoch [1/1000]  Training Loss:.. 0.095 Test Loss:.. 6.411 Test Accuracy:.. 0.003
Epoch [1/1000]  Training Loss:.. 0.095 Test Loss:.. 6.408 Test Accuracy:.. 0.000
Epoch [1/1000]  Training Loss:.. 0.094 Test Loss:.. 6.412 Test Accuracy:.. 0.002
Epoch [1/1000]  Training Loss:.. 0.094 Test Loss:.. 6.416 Test Accuracy:.. 0.002
Epoch [2/1000]  Training Loss:.. 0.096 Test Loss:.. 6.414 Test Accuracy:.. 0.002
Epoch [2/1000]  Training Loss:.. 0.094 Test Loss:.. 6.415 Test Accuracy:.. 0.002
Epoch [2/1000]  Training Loss:.. 0.094 Test Loss:.. 6.423 Test Accuracy:.. 0.002
Epoch [2/1000]  Training Loss:.. 0.094 Test Loss:.. 6.411 Test Accuracy:.. 0.002
Epoch [2/1000]  Training Loss:.. 0.107 Test Loss:.. 6.418 Test Accuracy:.. 0.002
Epoch [2/1000]  Training Loss:.. 0.094 Test Loss:.. 6.462 Test Accuracy:.. 0.000
Epoch [2/1000]  Training Los

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader: #-- note that we aren't able to test_loader here due to some errors
        images, labels = images.cuda(), labels.cuda()
        
        pred = model.forward(images)
        
        predictions = torch.argmax(pred.data, 1)
        
        correct += (predictions == labels).sum().item()
    total = test_labels.size()[0]

In [ ]:
print('training-accuracy:', (correct/total)*100)

In [ ]:
# testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader: #-- note that we aren't able to test_loader here due to some errors
        images, labels = images.cuda(), labels.cuda()
        
        pred = model.forward(images)
        
        predictions = torch.argmax(pred.data, 1)
        
        correct += (predictions == labels).sum().item()
    total = test_labels.size()[0]

In [ ]:
print('testing-accuracy:', (correct/total)*100)

In [ ]:
# save the model params checkpoint
torch.save(model.state_dict(), 'UFI-best-params.ckpt')